In [1]:
import os

import geojson
import numpy as np

In [2]:
threshold = .1

In [14]:
files = ['28_px_amazonas_NW_v7-2020'
        ]

for fn in files:
    with open('../outputs/' + fn + '.geojson', 'r') as f:
        data = geojson.load(f)

    preds = np.array([point['properties']['mean'] for point in data[:]])
    coords = np.array([np.mean(point['geometry']['coordinates'][0], axis=(0)) for point in data[:]])
    filtered = [sample for sample in data['features'] if sample['properties']['mean'] > threshold]
    filtered_fc = geojson.FeatureCollection(filtered)

    with open(f"../outputs/{fn}_thresh_{threshold}.geojson", 'w') as f:
        geojson.dump(filtered_fc, f)

    filtered_features = []

    for pred, coord in zip(preds, coords):
        if pred > threshold:
            point = geojson.Point((coord[0], coord[1]))
            filtered_features.append(geojson.Feature(geometry=point, properties={'pred': pred}))

    filtered_collection_points = geojson.FeatureCollection(filtered_features)
    print(len(filtered_features))
    with open(f"../outputs/{fn}_thresh_{threshold}_points.geojson", 'w') as f:
        geojson.dump(filtered_collection_points, f)

625


In [8]:
filtered_fc = geojson.FeatureCollection(filtered)
with open(f"../outputs/{fn}_thresh_{threshold}.geojson", 'w') as f:
    geojson.dump(filtered_fc, f)

In [ ]:
filtered_features = []

for pred, coord in zip(preds, coords):
    if pred > threshold:
        point = geojson.Point((coord[0], coord[1]))
        filtered_features.append(geojson.Feature(geometry=point, properties={'pred': pred}))
        
filtered_collection_points = geojson.FeatureCollection(filtered_features)
print(len(filtered_features))
with open(f"../outputs/{fn}_thresh_{threshold}_points.geojson", 'w') as f:
    geojson.dump(filtered_collection_points, f)

## Combine Tapajos Timeseries

In [39]:
filename = '28_px_tapajos_v7-'

series = []

for year in range(2016, 2021):
    with open('../outputs/' + filename + str(year) + '.geojson', 'r') as f:
        features = geojson.load(f)['features']
        for feature in features:
            if feature['properties']['mean'] > 0.5:
                feature['properties']['date'] = f"{year}/03/01 00:00"
                series.append(feature)

In [41]:
fc = geojson.FeatureCollection(series)
with open(f"../outputs/tapajos_time_series_2016-2020_thresh_0.5.geojson", 'w') as f:
    geojson.dump(fc, f)

## Combine Amazonas Preds

In [45]:
filename = '28_px_amazonas_'

series = []
for region in ['NE', 'SE', 'NW', 'SW']:
    fn = filename + region + '_v7-2020.geojson'
    with open('../outputs/' + fn, 'r') as f:
        features = geojson.load(f)['features']
        for feature in features:
            if feature['properties']['mean'] > 0.5:
                series.append(feature)
    
fc = geojson.FeatureCollection(series)
with open(f"../outputs/amazonas_2020_thresh_0.5.geojson", 'w') as f:
    geojson.dump(fc, f)

## Combine Bolivar Preds

In [11]:
import geojson
filename = '28_px_bolivar-'
threshold = 0.8
series = []
for region in range(1,7):
    fn = filename + str(region) + '_v9-2016.geojson'
    with open('../outputs/' + fn, 'r') as f:
        features = geojson.load(f)['features']
        for feature in features:
            if feature['properties']['mean'] > threshold:
                series.append(feature)
    
fc = geojson.FeatureCollection(series)
with open(f"../outputs/28_px_bolivar_v9_2016_thresh_{threshold}.geojson", 'w') as f:
    geojson.dump(fc, f)